# Element distribution interpolation

In this example we show how to perform interpolations that keep the same element structure.

Two examples of this are:

1. Mapping the GLL points of each element into a different distribution.
2. Perfomring P refinement, i.e., map to points of the same distribution but a different degree.

#### Import general modules

In [1]:
# Import required modules
from mpi4py import MPI #equivalent to the use of MPI_init() in C
import matplotlib.pyplot as plt
import numpy as np

# Get mpi info
comm = MPI.COMM_WORLD

#### Import modules from pynek

In [2]:
from pynektools.io.ppymech.neksuite import pynekread, pynekwrite
from pynektools.datatypes.msh import Mesh
from pynektools.datatypes.field import FieldRegistry

## Read the data and build objects

In this instance, we create connectivity for the mesh object, given that we wish to use direct stiffness summation to reduce discontinuities.

In [3]:
msh = Mesh(comm, create_connectivity=True)
fld = FieldRegistry(comm)
pynekread('../data/tc_channel0.f00001', comm, data_dtype=np.double, msh=msh, fld=fld)

2024-08-25 23:03:57,096 - Mesh - INFO - Initializing empty Mesh object.
2024-08-25 23:03:57,098 - Field - INFO - Initializing empty Field object
2024-08-25 23:03:57,099 - pynekread - INFO - Reading file: ../data/tc_channel0.f00001
2024-08-25 23:03:57,113 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2024-08-25 23:03:57,114 - Mesh - INFO - Initializing common attributes.
2024-08-25 23:03:57,114 - Mesh - INFO - Creating connectivity
2024-08-25 23:03:57,319 - Mesh - INFO - Mesh object initialized.
2024-08-25 23:03:57,320 - Mesh - INFO - Mesh data is of type: float64
2024-08-25 23:03:57,320 - Mesh - INFO - Elapsed time: 0.207341994s
2024-08-25 23:03:57,321 - pynekread - INFO - Reading field data
2024-08-25 23:03:57,329 - pynekread - INFO - File read
2024-08-25 23:03:57,330 - pynekread - INFO - Elapsed time: 0.23145508s


## Mapping to a new distribution

We can map to a new distribution of points in any of the dimensions of the element at the same time.

Here we show an example where this is done to an equidistant mesh in the z direction.

For this we use the PMapper class.

In [4]:
# Import the module
from pynektools.interpolation.mesh_to_mesh import PMapper

# Here specify that it should be equal in the 3 direction
mapper = PMapper(n=msh.lx, distribution=['GLL', 'GLL', 'EQ'])

# Create the mesh with the new distribution
eq_msh = mapper.get_new_mesh(comm, msh=msh)

# Interpolate the fields. They are passed as a list and returned as a list
mapped_fields = mapper.interpolate_from_field_list(comm, field_list=[fld.registry['u'],fld.registry['v'],fld.registry['w']])

# Create an empty field object for the equidistant fields
eq_fld = FieldRegistry(comm)
eq_fld.add_field(comm, field_name='u', field=mapped_fields[0], dtype = np.double)
eq_fld.add_field(comm, field_name='v', field=mapped_fields[1], dtype = np.double)
eq_fld.add_field(comm, field_name='w', field=mapped_fields[2], dtype = np.double)

# Write the new mesh and fields
fname = "mappedfield0.f00001"
pynekwrite(fname, comm, msh=eq_msh, fld=eq_fld, write_mesh=True, wdsz=4)

2024-08-25 23:03:57,919 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2024-08-25 23:03:57,920 - Mesh - INFO - Initializing common attributes.
2024-08-25 23:03:57,920 - Mesh - INFO - Creating connectivity
2024-08-25 23:03:58,145 - Mesh - INFO - Mesh object initialized.
2024-08-25 23:03:58,146 - Mesh - INFO - Mesh data is of type: float64
2024-08-25 23:03:58,146 - Mesh - INFO - Elapsed time: 0.2272013530000001s
2024-08-25 23:03:58,507 - Field - INFO - Initializing empty Field object
2024-08-25 23:03:58,508 - pynekwrite - INFO - Writing file: mappedfield0.f00001
2024-08-25 23:03:58,518 - pynekwrite - INFO - Elapsed time: 0.009620887999999939s


## Performing P refinement

In some instances one wishes to keep the GLL distribution to perform integration or derivation but doing so at higher or lower polynomiar order. For this cases one can employ p refinement/coarsening. We show now how that can be done.



In [5]:
# Import the module
from pynektools.interpolation.mesh_to_mesh import PRefiner

# Here specify that it should be equal in the 3 direction
pr = PRefiner(n_old = msh.lx, n_new = 3)

# Create the mesh with the new distribution
r_msh = pr.get_new_mesh(comm, msh=msh)

# Interpolate the fields. They are passed as a list and returned as a list
r_fields = pr.interpolate_from_field_list(comm, field_list=[fld.registry['u'],fld.registry['v'],fld.registry['w']])

# Create an empty field object for the equidistant fields
r_fld = FieldRegistry(comm)
r_fld.add_field(comm, field_name='u', field=r_fields[0], dtype = np.double)
r_fld.add_field(comm, field_name='v', field=r_fields[1], dtype = np.double)
r_fld.add_field(comm, field_name='w', field=r_fields[2], dtype = np.double)

# Write the new mesh and fields
fname = "refinedfield0.f00001"
pynekwrite(fname, comm, msh=r_msh, fld=r_fld, write_mesh=True, wdsz=4)

2024-08-25 23:03:58,879 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2024-08-25 23:03:58,880 - Mesh - INFO - Initializing common attributes.
2024-08-25 23:03:58,880 - Mesh - INFO - Creating connectivity
2024-08-25 23:03:58,905 - Mesh - INFO - Mesh object initialized.
2024-08-25 23:03:58,906 - Mesh - INFO - Mesh data is of type: float64
2024-08-25 23:03:58,906 - Mesh - INFO - Elapsed time: 0.027189482999999903s
2024-08-25 23:03:59,260 - Field - INFO - Initializing empty Field object
2024-08-25 23:03:59,261 - pynekwrite - INFO - Writing file: refinedfield0.f00001
2024-08-25 23:03:59,266 - pynekwrite - INFO - Elapsed time: 0.004223237000000157s
